In [30]:
from PIL import Image, ImageFont, ImageDraw 
import glob
import string
import random
import os
import shutil
import math
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras.layers import MaxPooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, Rescaling
from tensorflow.keras import Input 
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt 
import cv2

2.6.0


In [40]:
unmarked_path = "C:/Users/Scrip0/Desktop/Training/Unmarked" # Path to marked images
marked_path = os.path.dirname(unmarked_path) + "/Marked"
unmarked_imgs = []
marked_imgs = []
ext = ["jpg","png", "jpeg"];
for file in (file for file in (glob.glob(unmarked_path + '/*.%s' % e) for e in ext) if file != []):
    for image in file:
        unmarked_imgs.append(Image.open(image))

In [41]:
 unmarked_imgs

[<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=585x487 at 0x19FD551A4C0>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=585x487 at 0x19FD551A2B0>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=585x487 at 0x19FD59EF190>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=585x487 at 0x19FD59EF9D0>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=585x487 at 0x19FD59EF700>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=585x487 at 0x19FD59EF0A0>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=585x487 at 0x19FD59EFDC0>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=585x487 at 0x19FD59EF250>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=585x487 at 0x19FD59EF5B0>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=585x487 at 0x19FD59EFCD0>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=585x487 at 0x19FD59EFA00>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=585x487 at 0x19FD112F6D0>,
 <PIL.PngImagePl

In [42]:
def generate_word():
    letters = string.ascii_letters + string.digits + string.punctuation;
    return "".join(random.choice(letters) for i in range(random.randint(5, 20)))

In [43]:
class point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
def generate_points(width, height, angle, side):
    points = []
    num = random.randint(2, 5)
    half_width = int(width / 2)
    gap = int(height / (num - 1)) 
    if (side == "right"): gap = int(gap * 1.5)
    extra = random.randint(0, gap)
    for i in range(0, num):
        points.append(point(half_width, gap * i - extra))
        
    if (side == "right"):
        more = int(((-half_width * math.tan(math.radians(angle))) - extra) / gap)  + 1
    else:
        more = int(((half_width * math.tan(math.radians(angle))) - extra) / gap)  + 1
    for i in range(-more - 1, 0):
        points.insert(0, point(half_width, gap * i - extra))
    for i in range(num, num + more):
        points.append(point(half_width, gap * i - extra))
    return points

def draw_lines_left(image, angle, watermark, line, color, opacity, size):
    points = generate_points(image.size[0], image.size[1], angle, "left")
    half_width = int(image.width / 2)
    tan = math.tan(math.radians(angle))
    for point in points:
        x = 0
        y = int(half_width * tan + point.y)
        while (x < image.width):
            image.paste(watermark, (x, y), watermark)
            line = Image.new('RGBA', image.size, (255,255,255,0))
            draw = ImageDraw.Draw(line)
            line_width = size * 3
            draw.line((x + watermark.width, y, x + watermark.width + line_width, y - line_width * tan), fill=(color[0], color[1], color[2], opacity), width=int(size / 7))
            image.alpha_composite(line)
            x += watermark.width + line_width
            y -= int(line_width * tan) + watermark.height
            
def draw_lines_right(image, angle, watermark, line, color, opacity, size):
    points = generate_points(image.size[0], image.size[1], angle, "right")
    half_width = int(image.width / 2)
    tan = math.tan(math.radians(angle))
    for point in points:
        x = image.width - watermark.width
        y = int(-half_width * tan + point.y)
        while (x > -watermark.width):
            image.paste(watermark, (x, y), watermark)
            line = Image.new('RGBA', image.size, (255,255,255,0))
            draw = ImageDraw.Draw(line)
            line_width = size * 3
            draw.line((x, y, x - line_width, y + line_width * tan), fill=(color[0], color[1], color[2], opacity), width=int(size / 7))
            image.alpha_composite(line)
            x -= watermark.width + line_width
            y -= -int(line_width * tan) + watermark.height

In [44]:
if (os.path.isdir(marked_path)): shutil.rmtree(marked_path, ignore_errors=True)
os.mkdir(marked_path)
for img in unmarked_imgs:
    angle = random.randint(30, 60)
    color = random.randint(0, 256), random.randint(0, 256), random.randint(0, 256)
    opacity = random.randint(40, 250)
    image = img.convert('RGBA')
    text = generate_word()
    size = random.randint(10, 70)
    font = ImageFont.truetype("impact.ttf", size)
    
    watermark = Image.new('RGBA', font.getsize(text), (255,255,255,0))
    d = ImageDraw.Draw(watermark)    
    d.text((0, 0), text, fill=(color[0], color[1], color[2], opacity), font=font)
    
    watermark = watermark.rotate(angle, expand = 1)
    draw_lines_left(image, angle, watermark, image, color, opacity, size)
    
    watermark = watermark.rotate(-90, expand = 1)
    draw_lines_right(image, -90 + angle, watermark, image, color, opacity, size)
    path = marked_path + "/" + os.path.basename(img.filename)
    image.save(path)
    
    marked_imgs.append(path)

In [45]:

# plt.imshow(np.array(marked_imgs[0]))
# plt.show()

In [46]:
width = 200
height = 200
def createPixelArr(files):
    data = []
    for image in files:# take each image and use imread to get the pixel values in a matrix 
            img_arr = cv2.imread(image, cv2.IMREAD_COLOR)
            resized_arr = cv2.resize(img_arr, (width, height)) # rescale the image so every image is of the same dimension
            data.append(resized_arr) # add the matrix of pixel values 
    return np.array(data)

def createPaths(images):
    paths = []
    for image in images:
        paths.append(image.filename)
    return paths

In [47]:
train_marked = createPixelArr(marked_imgs)
train_unmarked = createPixelArr(createPaths(unmarked_imgs))

In [48]:
x_train, x_test, y_train, y_test = train_test_split(train_marked, train_unmarked, train_size=0.8, random_state=1) 